In [11]:
# Importing Dependencies

import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.callbacks import EarlyStopping
import matplotlib as plt

import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png

print('Imports Done')

Imports Done


In [2]:
# paths to training and validation data

train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/test'

In [3]:
# Params for CNN

img_width, img_height = 150, 150
batch_size = 5
epochs = 200
train_samples = 420
validation_samples = 80

In [4]:
# Loading vgg except the final layer

vgg_conv = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [5]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [6]:
model_top = Sequential()
# Add the vgg convolutional base model
model_top.add(vgg_conv)

model_top.add(Flatten())
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.2))
model_top.add(Dense(2, activation='sigmoid'))
model_top.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_top.summary()

model_to_png(model_top, 'Fine_Tuned')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
__________

In [8]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_batchsize = 210
val_batchsize = 40
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=train_batchsize,
        class_mode='categorical') # class_mode='categorical'
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=val_batchsize,
        class_mode='categorical', # class_mode='categorical'
        shuffle=False)


Found 420 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [9]:
#Compile the model
model_top.compile(loss='binary_crossentropy', #categorical_crossentropy
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# define the checkpoint
filepath = "model_finetuned.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=8, verbose=1, mode='max')
callbacks_list = [checkpoint, early_stopping]

#Train the model
history = model_top.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=60,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
      callbacks=callbacks_list)

Epoch 1/60
2/2 [==============================] - 9s 5s/step - loss: 0.7475 - acc: 0.5036 - val_loss: 0.6754 - val_acc: 0.5750

Epoch 00001: val_acc improved from -inf to 0.57500, saving model to model_finetuned.h5
Epoch 2/60
2/2 [==============================] - 3s 2s/step - loss: 0.6841 - acc: 0.5524 - val_loss: 0.6221 - val_acc: 0.7000

Epoch 00002: val_acc improved from 0.57500 to 0.70000, saving model to model_finetuned.h5
Epoch 3/60
2/2 [==============================] - 4s 2s/step - loss: 0.5828 - acc: 0.6833 - val_loss: 0.5588 - val_acc: 0.7188

Epoch 00003: val_acc improved from 0.70000 to 0.71875, saving model to model_finetuned.h5
Epoch 4/60
2/2 [==============================] - 4s 2s/step - loss: 0.7632 - acc: 0.6393 - val_loss: 0.6279 - val_acc: 0.6812

Epoch 00004: val_acc did not improve
Epoch 5/60
2/2 [==============================] - 4s 2s/step - loss: 0.5867 - acc: 0.6833 - val_loss: 0.5722 - val_acc: 0.7437

Epoch 00005: val_acc improved from 0.71875 to 0.74375, s

In [10]:
model_top.save_weights('fine-tined-8812.h5')

In [15]:
prediction_data_dir = 'dataset/predict'
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        prediction_data_dir,
        target_size=(img_width, img_height),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 12 images belonging to 2 classes.


In [17]:
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model_top.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

1/0 [====================================================================================================] - 1s 1s/step
No of errors = 0/12


In [18]:
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()